In [ ]:
# =========================================================
# PhysioNet ECG Image Digitization — v4m+ (safe template-first blend)
# ResNet34-H + FiLM + Temporal Head  (3-channel input)
# ROI: heuristic(train) + UNet(train/val), test = grid-only (stable)
# Tilt-aware TTA, multi-scale freq loss, cosine LR, motion/perspective augs
# Safe blend: lag-aligned + sign-check; template default; model weight gated by NCC
# Output: /kaggle/working/submission.parquet  &  submission.csv
# Includes: submission validator + offline LB estimate (official metric)
# =========================================================
import os, gc, cv2, glob, math, random, warnings, pickle
warnings.filterwarnings("ignore")
import numpy as np, pandas as pd
from tqdm import tqdm
import scipy.signal, scipy.optimize

import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# ---------------------------
# Config (safer blending + longer train)
# ---------------------------
class CFG:
    DATA_DIR = "/kaggle/input/physionet-ecg-image-digitization"
    SEED = 1337
    IMG_H = 352
    IMG_W = 1280
    T_TRAIN = 1536
    BATCH_SIZE = 2
    EPOCHS_MAIN = 10                 # ↑ train a bit longer
    LR = 3e-4
    WD = 1e-4
    NUM_WORKERS = 2
    AMP = True
    CHANNELS_LAST = True
    VALID_FRAC = 0.05
    SAVE_BEST = True

    # ROI modes
    ROI_MODE_TRAIN = "heuristic"
    ROI_MODE_VAL   = "unet"
    ROI_MODE_TESTS = ["grid"]        # ← safest & most stable for LB

    GRID_USE_RHYTHM_FOR_II = True
    TRAIN_ROI_UNET = True
    ROI_EPOCHS = 1
    ROI_IN_TRAIN = True
    LEAD_BAND_FRAC = 0.06
    NMS_MIN_DIST_FRAC = 0.04
    ROI_BAND_MARGIN = 0.015

    # Augs
    AUG_BRIGHTNESS = 0.12
    AUG_CONTRAST = 0.12
    AUG_ROT_DEG = 0.7
    AUG_PERSPECTIVE = 0.015
    AUG_MOTION_BLUR_P = 0.25

    # Inference
    TTA_BASE_DEGS = [-0.4, 0.0, 0.4]
    TTA_EXTRA = True

    # Filters (we’ll fix cutoff to 15 Hz)
    BANDPASS_LOW = 0.5
    BANDPASS_HIGH = 40.0
    BANDPASS_ORDER = 3

    # Templates
    TEMPLATE_LEN = 500
    TEMPLATE_MAX_SAMPLES = 4000
    TEMPLATE_CACHE = "/kaggle/working/lead_templates.pkl"

    # Blend (safe: template default)
    ENSEMBLE_BETA_BASE = 0.0         # ← no unconditional model weight
    ENSEMBLE_BETA_GAIN = 0.50        # scales with correlation
    ENSEMBLE_BETA_CLIP = (0.00, 0.65)

    OUT_MIN = 0.00
    OUT_MAX = 0.09                   # ← match stronger baseline

    # Train knobs
    GRAD_ACCUM_TARGET = 8            # larger effective batch

    # Local sanity
    DO_LOCAL_SNR = True

def set_seed(seed=1337):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
set_seed(CFG.SEED)

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.benchmark = True

train_csv = pd.read_csv(f"{CFG.DATA_DIR}/train.csv")
test_csv  = pd.read_csv(f"{CFG.DATA_DIR}/test.csv")
LEADS = ['I','II','III','aVR','aVL','aVF','V1','V2','V3','V4','V5','V6']
LEAD_TO_IDX = {l:i for i,l in enumerate(LEADS)}
VARIANTS = ["0001","0003","0004","0005","0006","0009","0010","0011","0012"]
print("Device:", device, "GPUs:", torch.cuda.device_count())
print("Train records:", len(train_csv), " Test records:", len(test_csv))

# ---------------------------
# IO / preprocess
# ---------------------------
def pick_image_path(base_id, prefer_first=True):
    folder = f"{CFG.DATA_DIR}/train/{base_id}"
    if not os.path.isdir(folder): return None
    paths = [f"{folder}/{base_id}-{s}.png" for s in VARIANTS if os.path.exists(f"{folder}/{base_id}-{s}.png")]
    if not paths: paths = glob.glob(f"{folder}/*.png")
    if not paths: return None
    return paths[0] if prefer_first else random.choice(paths)

def apply_motion_blur(img01):
    if np.random.rand() > CFG.AUG_MOTION_BLUR_P: return img01
    k = np.random.choice([3,5,7])
    kernel = np.zeros((k, k), dtype=np.float32)
    idx = k//2
    kernel[idx, :] = 1.0 / k
    return cv2.filter2D(img01, -1, kernel)

def perspective_jitter(img01):
    h,w = img01.shape
    d = CFG.AUG_PERSPECTIVE
    if d <= 1e-6: return img01
    dx, dy = d*w, d*h
    src = np.float32([[0,0],[w,0],[w,h],[0,h]])
    dst = np.float32([
        [np.random.uniform(-dx, dx), np.random.uniform(-dy, dy)],
        [w+np.random.uniform(-dx, dx), np.random.uniform(-dy, dy)],
        [w+np.random.uniform(-dx, dx), h+np.random.uniform(-dy, dy)],
        [np.random.uniform(-dx, dx), h+np.random.uniform(-dy, dy)]
    ])
    M = cv2.getPerspectiveTransform(src, dst)
    out = cv2.warpPerspective((img01*255).astype(np.uint8), M, (w,h),
                              flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)
    return out.astype(np.float32)/255.0

def load_img_gray(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (CFG.IMG_W, CFG.IMG_H), interpolation=cv2.INTER_AREA)
    img = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8)).apply(img)
    return img.astype(np.float32) / 255.0

def rotate_image(img01, deg):
    if abs(deg) < 1e-6: return img01
    M = cv2.getRotationMatrix2D((CFG.IMG_W/2, CFG.IMG_H/2), deg, 1.0)
    rot = cv2.warpAffine((img01*255).astype(np.uint8), M, (CFG.IMG_W, CFG.IMG_H),
                         flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)
    return rot.astype(np.float32)/255.0

def to_three_channel(img01):
    sx = cv2.Sobel(img01, cv2.CV_32F, 1, 0, ksize=3)
    sy = cv2.Sobel(img01, cv2.CV_32F, 0, 1, ksize=3)
    sx = (sx - sx.min()) / (sx.max() - sx.min() + 1e-6)
    sy = (sy - sy.min()) / (sy.max() - sy.min() + 1e-6)
    return np.stack([img01, sx, sy], axis=0)  # (3,H,W)

def random_augment(img01):
    if CFG.AUG_BRIGHTNESS>0:
        b=np.random.uniform(-CFG.AUG_BRIGHTNESS, CFG.AUG_BRIGHTNESS); img01=np.clip(img01+b,0,1)
    if CFG.AUG_CONTRAST>0:
        c=np.random.uniform(1-CFG.AUG_CONTRAST,1+CFG.AUG_CONTRAST); m=img01.mean(); img01=np.clip((img01-m)*c+m,0,1)
    if CFG.AUG_ROT_DEG>0:
        deg=np.random.uniform(-CFG.AUG_ROT_DEG, CFG.AUG_ROT_DEG); img01=rotate_image(img01,deg)
    img01 = perspective_jitter(img01)
    img01 = apply_motion_blur(img01)
    return img01

def resample_1d(x, new_len):
    new_len = int(new_len)
    if new_len <= 0:
        return np.zeros(0, dtype=np.float32)
    x = np.asarray(x, dtype=np.float32).reshape(-1)
    n = x.shape[0]
    if n == 0:
        return np.zeros(new_len, dtype=np.float32)
    if n == 1:
        return np.full(new_len, float(x[0]), dtype=np.float32)
    xp = np.linspace(0.0, n - 1.0, num=n, dtype=np.float64)
    fp = np.nan_to_num(x, nan=0.0, posinf=0.0, neginf=0.0).astype(np.float64)
    xnew = np.linspace(0.0, n - 1.0, num=new_len, dtype=np.float64)
    y = np.interp(xnew, xp, fp).astype(np.float32)
    return y

# Tiny tilt estimator (for grid-aligned pages)
def estimate_tilt_deg(img01):
    im = (img01*255).astype(np.uint8)
    edges = cv2.Canny(im, 50, 150, apertureSize=3)
    lines = cv2.HoughLines(edges, 1, np.pi/180, threshold=200)
    if lines is None: return 0.0
    angs=[]
    for rho,theta in lines[:,0,:]:
        a = (theta - np.pi/2) * 180/np.pi  # horizontal grid → near 0°
        if -3.0 <= a <= 3.0: angs.append(a)
    if not angs: return 0.0
    a = float(np.median(angs))
    return float(np.clip(a, -1.2, 1.2))

# ---------------------------
# ROI helpers (heuristic / UNet / grid)
# ---------------------------
def heuristic_peak_bands(img01):
    h,w=img01.shape
    edges=cv2.Canny((img01*255).astype(np.uint8),40,120)
    proj=edges.sum(axis=1).astype(np.float32)
    p=(proj-proj.min())/(proj.max()-proj.min()+1e-6)
    min_dist=max(1,int(CFG.NMS_MIN_DIST_FRAC*h))
    peaks=[]
    for y in range(h):
        l=max(0,y-min_dist); r=min(h,y+min_dist+1)
        if p[y]==p[l:r].max() and p[y]>0.1: peaks.append((p[y],y))
    peaks=sorted(peaks,key=lambda t:t[0],reverse=True)
    bands=[]; used=np.zeros(h,bool); band_half=int(CFG.LEAD_BAND_FRAC*h/2)
    for _,y in peaks:
        if used[max(0,y-min_dist):min(h,y+min_dist+1)].any(): continue
        y0=max(0,y-band_half-int(CFG.ROI_BAND_MARGIN*h)); y1=min(h,y+band_half+int(CFG.ROI_BAND_MARGIN*h))
        bands.append((y0,y1)); used[y0:y1]=True
        if len(bands)==12: break
    bands=sorted(bands,key=lambda b:(b[0]+b[1])/2)
    while len(bands)<12: bands.append((int(len(bands)*h/12), int((len(bands)+1)*h/12)))
    return bands[:12]

def crops_from_heuristic(img01):
    return [img01[y0:y1,:] for (y0,y1) in heuristic_peak_bands(img01)]

def stack_crop_to_x3(crop):
    Hc=max(48,int(CFG.LEAD_BAND_FRAC*CFG.IMG_H))
    c=cv2.resize(crop,(CFG.IMG_W,Hc),interpolation=cv2.INTER_AREA)
    return to_three_channel(c)  # (3,Hc,W)

class TinyUNet(nn.Module):
    def __init__(self,in_ch=1,out_ch=12,base=16):
        super().__init__()
        self.e1=nn.Sequential(nn.Conv2d(in_ch,base,3,1,1),nn.ReLU(True),nn.Conv2d(base,base,3,1,1),nn.ReLU(True))
        self.e2=nn.Sequential(nn.Conv2d(base,base*2,3,2,1),nn.ReLU(True))
        self.e3=nn.Sequential(nn.Conv2d(base*2,base*4,3,2,1),nn.ReLU(True))
        self.d2=nn.Sequential(nn.ConvTranspose2d(base*4,base*2,2,2),nn.ReLU(True))
        self.d1=nn.Sequential(nn.ConvTranspose2d(base*2,base,2,2),nn.ReLU(True))
        self.outc=nn.Conv2d(base,out_ch,1)
    def forward(self,x):
        x1=self.e1(x); x2=self.e2(x1); x3=self.e3(x2)
        y=self.d2(x3); y=self.d1(y); return self.outc(y)

def crops_from_mask(img01, mask12):
    H,W=img01.shape; crops=[]
    for i in range(12):
        m=mask12[i]; col=m.mean(axis=1)
        y=int(np.clip(np.average(np.arange(H),weights=(col+1e-6)),0,H-1))
        band=max(4,int(CFG.LEAD_BAND_FRAC*H)); y0=max(0,y-band//2); y1=min(H,y0+band)
        if y1-y0<band: y0=max(0,y1-band)
        crops.append(img01[y0:y1,:])
    return crops

GRID_ROWS = [
    ['I','II','III'],
    ['aVR','aVL','aVF'],
    ['V1','V2','V3'],
    ['V4','V5','V6'],
]
def crops_from_grid(img01, use_rhythm_for_ii=True):
    H, W = img01.shape
    rhythm_h = int(0.22 * H); rhythm_h = max(int(0.15*H), min(int(0.30*H), rhythm_h))
    grid_h = H - rhythm_h
    row_edges = np.linspace(0, grid_h, 5).astype(int)
    col_edges = np.linspace(0, W, 4).astype(int)
    lead_to_crop = {}
    for r in range(4):
        y0, y1 = row_edges[r], row_edges[r+1]
        for c in range(3):
            x0, x1 = col_edges[c], col_edges[c+1]
            lead = GRID_ROWS[r][c]
            lead_to_crop[lead] = img01[y0:y1, x0:x1]
    rhythm = img01[grid_h:H, :]
    if use_rhythm_for_ii:
        lead_to_crop['II'] = rhythm
    return [lead_to_crop[l] for l in LEADS]

def get_crops(img01, roi_mode="heuristic", roi_net=None):
    if roi_mode == "unet" and (roi_net is not None):
        with torch.no_grad():
            # CPU tensor; DO NOT .to(device) in a worker
            x = torch.from_numpy(img01).unsqueeze(0).unsqueeze(0).float()
            if CFG.CHANNELS_LAST:
                x = x.to(memory_format=torch.channels_last)
            m = roi_net(x).sigmoid()[0].detach().numpy()
        return crops_from_mask(img01, m)
    elif roi_mode == "grid":
        return crops_from_grid(img01, use_rhythm_for_ii=CFG.GRID_USE_RHYTHM_FOR_II)
    else:
        return crops_from_heuristic(img01)
        
# ---------------------------
# Model: ResNet34-H + FiLM + Temporal
# ---------------------------
class BasicBlockH(nn.Module):
    expansion=1
    def __init__(self,inplanes,planes,stride=(1,1),downsample=None):
        super().__init__()
        self.conv1=nn.Conv2d(inplanes,planes,3,stride=stride,padding=1,bias=False)
        self.bn1=nn.BatchNorm2d(planes); self.relu=nn.ReLU(True)
        self.conv2=nn.Conv2d(planes,planes,3,1,1,bias=False); self.bn2=nn.BatchNorm2d(planes)
        self.downsample=downsample
    def forward(self,x):
        idt=x
        out=self.relu(self.bn1(self.conv1(x)))
        out=self.bn2(self.conv2(out))
        if self.downsample is not None: idt=self.downsample(x)
        return self.relu(out+idt)

def _make_layer_h(inplanes,planes,blocks,stride_h):
    down=None
    if stride_h!=1 or inplanes!=planes:
        down=nn.Sequential(nn.Conv2d(inplanes,planes,1,(stride_h,1),bias=False), nn.BatchNorm2d(planes))
    layers=[BasicBlockH(inplanes,planes,(stride_h,1),down)]
    for _ in range(1,blocks): layers.append(BasicBlockH(planes,planes,(1,1)))
    return nn.Sequential(*layers)

class ResNet34H(nn.Module):
    def __init__(self,in_ch=3):   # ← 3 channels
        super().__init__()
        self.conv1=nn.Conv2d(in_ch,64,7,(2,1),3,bias=False)
        self.bn1=nn.BatchNorm2d(64); self.relu=nn.ReLU(True)
        self.maxpool=nn.MaxPool2d((3,3),(2,1),1)
        self.layer1=_make_layer_h(64,64,3,1)
        self.layer2=_make_layer_h(64,128,4,2)
        self.layer3=_make_layer_h(128,256,6,2)
        self.layer4=_make_layer_h(256,512,3,2)
        self.proj=nn.Conv2d(512,256,1)
    def forward(self,x):
        x=self.relu(self.bn1(self.conv1(x))); x=self.maxpool(x)
        x=self.layer1(x); x=self.layer2(x); x=self.layer3(x); x=self.layer4(x)
        x=self.proj(x).mean(dim=2)  # (B,256,W)
        return x

class DilatedTemporalBlock(nn.Module):
    def __init__(self,c,d):
        super().__init__(); self.conv=nn.Conv1d(c,c,3,padding=d,dilation=d,bias=False)
        self.bn=nn.BatchNorm1d(c); self.act=nn.ReLU(True)
    def forward(self,x): return x + self.act(self.bn(self.conv(x)))

class TemporalHead(nn.Module):
    def __init__(self,c=256,h=128):
        super().__init__()
        self.pre=nn.Sequential(nn.Conv1d(c,c,7,padding=3,bias=False), nn.BatchNorm1d(c), nn.ReLU(True))
        self.d1=DilatedTemporalBlock(c,1); self.d2=DilatedTemporalBlock(c,2)
        self.d4=DilatedTemporalBlock(c,4); self.d8=DilatedTemporalBlock(c,8)
        self.rnn=nn.GRU(c,h,batch_first=True,bidirectional=True)
        self.fuse=nn.Conv1d(2*h,c,1)
    def forward(self,x):
        x=self.pre(x); x=self.d1(x); x=self.d2(x); x=self.d4(x); x=self.d8(x)
        seq=x.transpose(1,2); self.rnn.flatten_parameters()
        y,_=self.rnn(seq); return self.fuse(y.transpose(1,2))

class FiLMLead(nn.Module):
    def __init__(self,n=12,d=64,c=256):
        super().__init__(); self.emb=nn.Embedding(n,d); self.g=nn.Linear(d,c); self.b=nn.Linear(d,c)
    def forward(self,x,lead_idx):
        e=self.emb(lead_idx); gamma=self.g(e).unsqueeze(-1); beta=self.b(e).unsqueeze(-1)
        return x*(1+gamma)+beta

class Head1D(nn.Module):
    def __init__(self,c=256): super().__init__(); self.net=nn.Sequential(nn.Conv1d(c,64,3,1,1), nn.ReLU(True), nn.Conv1d(64,1,3,1,1))
    def forward(self,x): return self.net(x).squeeze(1)

class ECGNetV4M(nn.Module):
    def __init__(self,in_ch=3):
        super().__init__(); self.enc=ResNet34H(in_ch); self.tem=TemporalHead(); self.film=FiLMLead(); self.head=Head1D()
    def forward(self,x3,lead_idx):
        f=self.enc(x3); t=self.tem(f); t=self.film(t,lead_idx); return self.head(t)

# --- robust FFT + multiscale
def fft_mag(x: torch.Tensor) -> torch.Tensor:
    with torch.cuda.amp.autocast(enabled=False):
        x32 = x.float()
        X = torch.fft.rfft(x32, dim=-1)
        mag = torch.abs(X)
        return mag

def ms_downsample(x, k):
    # avgpool downsample by k along last dim
    return F.avg_pool1d(x.unsqueeze(1), kernel_size=k, stride=k, ceil_mode=False).squeeze(1)

# --- safer CSV loader ---
def load_wave_csv_matrix(path):
    df = pd.read_csv(path)
    vals = []
    for c in LEADS:
        v = pd.to_numeric(df[c], errors='coerce').values.astype(np.float32)
        v = np.nan_to_num(v, nan=0.0, posinf=0.0, neginf=0.0)
        vals.append(v)
    arr = np.stack(vals, axis=0)
    return arr

def first_deriv(x): return x[...,1:]-x[...,:-1]

def main_loss(pred,target):
    # base L1 + derivative + multiscale spectral L1 (three scales)
    T=min(pred.shape[-1],target.shape[-1]); p=pred[...,:T]; t=target[...,:T]
    l = F.l1_loss(p,t) + 0.05*F.l1_loss(first_deriv(p),first_deriv(t))
    l += 0.10*F.l1_loss(fft_mag(p), fft_mag(t))
    p2,t2 = ms_downsample(p,2), ms_downsample(t,2)
    p4,t4 = ms_downsample(p,4), ms_downsample(t,4)
    l += 0.06*F.l1_loss(fft_mag(p2), fft_mag(t2))
    l += 0.04*F.l1_loss(fft_mag(p4), fft_mag(t4))
    return l

# ---------------------------
# Dataset
# ---------------------------
class LeadDataset(Dataset):
    def __init__(self, ids, roi_net=None, roi_mode="heuristic", augment=False, mix_unet=False):
        self.ids = list(ids); self.roi_net = roi_net; self.roi_mode = roi_mode; self.augment = augment
        self.mix_unet = mix_unet
        self.samples = [(id_, li) for id_ in self.ids for li in range(12)]
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        base_id, li = self.samples[idx]
        ipath = pick_image_path(base_id, prefer_first=True)
        if ipath is None: raise FileNotFoundError(f"no image for {base_id}")
        img01 = load_img_gray(ipath)
        if self.augment: img01 = random_augment(img01)
        # occasionally use UNet ROI even during "heuristic" train
        roi_mode_eff = self.roi_mode
        if self.mix_unet and (self.roi_net is not None) and (np.random.rand() < 0.5):
            roi_mode_eff = "unet"
        crops = get_crops(img01, roi_mode=roi_mode_eff, roi_net=self.roi_net if roi_mode_eff=="unet" else None)
        crop = crops[li]
        x3 = stack_crop_to_x3(crop)                      # (3,Hc,W)
        csv_path=f"{CFG.DATA_DIR}/train/{base_id}/{base_id}.csv"
        M=load_wave_csv_matrix(csv_path)
        y=resample_1d(M[li], CFG.T_TRAIN)
        y = np.nan_to_num(y, nan=0.0, posinf=0.0, neginf=0.0)
        return (torch.tensor(x3, dtype=torch.float32),
                torch.tensor(y, dtype=torch.float32),
                torch.tensor(li, dtype=torch.long))

# ---------------------------
# Metric (exact) — used for quick local sanity on Lead II
# ---------------------------
MAX_TIME_SHIFT = 0.2; PERFECT_SCORE = 384
class ParticipantVisibleError(Exception): pass
def compute_power_metric(label, prediction):
    if label.ndim!=1 or prediction.ndim!=1: raise ParticipantVisibleError('Inputs must be 1D.')
    finite_mask=np.isfinite(prediction)
    if not np.any(finite_mask): raise ParticipantVisibleError("prediction all NaN/inf")
    prediction=prediction.copy(); prediction[~np.isfinite(prediction)]=0
    noise=label-prediction; return np.sum(label**2), np.sum(noise**2)
def compute_snr_metric(signal, noise):
    if noise==0: return PERFECT_SCORE
    if signal==0: return 0
    return min((signal/noise), PERFECT_SCORE)
def align_signals_metric(label,pred,max_shift=float('inf')):
    if np.any(~np.isfinite(label)): raise ParticipantVisibleError('label finite')
    if np.sum(np.isfinite(pred))==0: raise ParticipantVisibleError('prediction finite')
    la=np.asarray(label,dtype=np.float64); pr=np.asarray(pred,dtype=np.float64)
    lc=la-np.mean(la); pc=pr-np.mean(pr)
    corr=scipy.signal.correlate(lc,pc,mode='full'); nl=la.size; npd=pr.size
    lags=scipy.signal.correlation_lags(nl,npd,mode='full'); valid=(lags>=-max_shift)&(lags<=max_shift)
    maxc=np.nanmax(corr[valid]); allmax=np.flatnonzero(corr==maxc); best=min(allmax,key=lambda i:abs(lags[i])); shift=lags[best]
    sp=max(shift,0); s=max(-shift,0); e=min(nl-shift,npd); ep=max(nl-npd-shift,0)
    aligned=np.concatenate((np.full(sp,np.nan), pr[s:e], np.full(ep,np.nan)))
    def obj(v): return np.nansum((la-(aligned-v))**2)
    if np.any(np.isfinite(la)&np.isfinite(aligned)):
        res=scipy.optimize.minimize_scalar(obj,method='Brent'); aligned-=res.x
    return aligned
def _calc_image_score(group):
    ufs=group['fs'].unique()
    if len(ufs)!=1: raise ParticipantVisibleError('fs consistent')
    fs=ufs[0]
    if fs!=int(len(group[group['lead']=='II'])/10): raise ParticipantVisibleError('len check')
    sum_sig=sum_noise=0.0
    for lead in LEADS:
        sub=group[group['lead']==lead]
        label=sub['value_true'].values; pred=sub['value_pred'].values
        aligned=align_signals_metric(label,pred,int(fs*MAX_TIME_SHIFT))
        ps,pn=compute_power_metric(label,aligned); sum_sig+=ps; sum_noise+=pn
    return compute_snr_metric(sum_sig,sum_noise)
def competition_score(solution, submission, row_id_column_name="id"):
    for df in [solution,submission]:
        if row_id_column_name not in df.columns: raise ParticipantVisibleError(f"'{row_id_column_name}' missing")
        if df['value'].isna().any(): raise ParticipantVisibleError('NaN in dfs')
        if not np.isfinite(df['value']).all(): raise ParticipantVisibleError('Inf in dfs')
    sub=submission[['id','value']]
    m=pd.merge(solution, sub, on=row_id_column_name, suffixes=('_true','_pred'))
    m['image_id']=m[row_id_column_name].str.split('_').str[0]
    m['row_id']=m[row_id_column_name].str.split('_').str[1].astype('int64')
    m['lead']=m[row_id_column_name].str.split('_').str[2]
    m.sort_values(by=['image_id','row_id','lead'], inplace=True)
    img_scores=m.groupby('image_id').apply(_calc_image_score, include_groups=False)
    return max(float(10*np.log10(img_scores.mean())), -PERFECT_SCORE)

# ---------------------------
# ROI quick train (small subset)
# ---------------------------
roi_net = None
if CFG.TRAIN_ROI_UNET:
    roi_net = TinyUNet().to(device)
    if CFG.CHANNELS_LAST:
        roi_net = roi_net.to(memory_format=torch.channels_last)
    opt_roi = torch.optim.AdamW(roi_net.parameters(), lr=1e-3, weight_decay=1e-4)
    scaler_roi = torch.cuda.amp.GradScaler(enabled=CFG.AMP)

    def build_pseudo_mask(img01: np.ndarray) -> np.ndarray:
        H, W = img01.shape
        mask = np.zeros((12, H, W), dtype=np.float32)
        for k, (y0, y1) in enumerate(heuristic_peak_bands(img01)):
            if k >= 12: break
            mask[k, y0:y1, :] = 1.0
        if mask.shape[0] < 12:
            pad = np.zeros((12 - mask.shape[0], H, W), dtype=np.float32)
            mask = np.concatenate([mask, pad], axis=0)
        return mask

    ids_all = train_csv["id"].values.copy()
    np.random.shuffle(ids_all)
    ids_small = ids_all[:max(200, int(0.15 * len(ids_all)))]
    bs = 8

    for ep in range(1, CFG.ROI_EPOCHS + 1):
        roi_net.train()
        losses = []
        pbar = tqdm(range(0, len(ids_small), bs), desc=f"ROI ep{ep}")
        for i in pbar:
            batch_ids = ids_small[i:i+bs]
            imgs, masks = [], []
            for bid in batch_ids:
                p = pick_image_path(bid, prefer_first=True)
                if p is None: continue
                img01 = load_img_gray(p)
                imgs.append(img01)
                masks.append(build_pseudo_mask(img01))
            if not imgs: continue

            x = torch.tensor(np.stack(imgs)[:, None, ...],
                             dtype=torch.float16 if CFG.AMP else torch.float32,
                             device=device)
            y = torch.tensor(np.stack(masks),
                             dtype=torch.float16 if CFG.AMP else torch.float32,
                             device=device)
            if CFG.CHANNELS_LAST:
                x = x.to(memory_format=torch.channels_last)

            opt_roi.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=CFG.AMP):
                logits = roi_net(x)
                loss = F.binary_cross_entropy_with_logits(logits, y)
            scaler_roi.scale(loss).backward()
            scaler_roi.step(opt_roi)
            scaler_roi.update()

            losses.append(loss.item())
            pbar.set_postfix(loss=np.mean(losses))
        print(f"ROI epoch {ep} loss={np.mean(losses):.4f}")

    torch.cuda.empty_cache()

if roi_net is not None:
    roi_net.eval()
    roi_net = roi_net.to('cpu')   # <<< important: avoid CUDA in worker

# ---------------------------
# Train main model
# ---------------------------
model=ECGNetV4M().to(device)
if CFG.CHANNELS_LAST: model=model.to(memory_format=torch.channels_last)
opt=torch.optim.AdamW(model.parameters(), lr=CFG.LR, weight_decay=CFG.WD)
# cosine LR
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=CFG.EPOCHS_MAIN, eta_min=CFG.LR*0.05)
scaler=torch.cuda.amp.GradScaler(enabled=CFG.AMP)

train_ids, val_ids = train_test_split(train_csv["id"].values, test_size=CFG.VALID_FRAC, random_state=CFG.SEED)
train_ds=LeadDataset(train_ids, roi_net=roi_net if CFG.ROI_IN_TRAIN else None,
                     roi_mode=CFG.ROI_MODE_TRAIN, augment=True, mix_unet=CFG.ROI_IN_TRAIN)
val_roi_mode = "unet" if roi_net is not None else "grid"
val_ds  =LeadDataset(val_ids,   roi_net=roi_net, roi_mode=val_roi_mode, augment=False)

train_loader=DataLoader(train_ds, batch_size=CFG.BATCH_SIZE, shuffle=True,
                        num_workers=CFG.NUM_WORKERS, pin_memory=True, persistent_workers=False)
val_loader = DataLoader(val_ds, batch_size=CFG.BATCH_SIZE, shuffle=False,
                        num_workers=0, pin_memory=False, persistent_workers=False)

accum_steps=max(1, CFG.GRAD_ACCUM_TARGET//CFG.BATCH_SIZE)

def train_one_epoch():
    model.train()
    losses = []
    opt.zero_grad(set_to_none=True)
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc="Train", leave=False)
    for step, (x3, y, li) in pbar:
        x3 = x3.to(device, non_blocking=True)
        y  = y.to(device,  non_blocking=True)
        li = li.to(device, non_blocking=True)
        if CFG.CHANNELS_LAST:
            x3 = x3.to(memory_format=torch.channels_last)
        with torch.cuda.amp.autocast(enabled=CFG.AMP):
            pred = model(x3, li)
            if pred.shape[-1] != CFG.T_TRAIN:
                pred = F.interpolate(pred.unsqueeze(1), size=CFG.T_TRAIN, mode="linear", align_corners=False).squeeze(1)
        pred = torch.nan_to_num(pred, nan=0.0, posinf=0.0, neginf=0.0)
        y    = torch.nan_to_num(y,    nan=0.0, posinf=0.0, neginf=0.0)
        with torch.cuda.amp.autocast(enabled=False):
            loss = main_loss(pred.float(), y.float()) / accum_steps
        scaler.scale(loss).backward()
        if (step + 1) % accum_steps == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(opt); scaler.update(); opt.zero_grad(set_to_none=True)
        losses.append(loss.item() * accum_steps)
        pbar.set_postfix(loss=float(np.mean(losses)))
    scheduler.step()
    return float(np.mean(losses))

@torch.no_grad()
def valid_epoch():
    model.eval()
    losses = []
    for x3, y, li in val_loader:
        x3 = x3.to(device, non_blocking=True)
        y  = y.to(device,  non_blocking=True)
        li = li.to(device, non_blocking=True)
        if CFG.CHANNELS_LAST:
            x3 = x3.to(memory_format=torch.channels_last)
        with torch.cuda.amp.autocast(enabled=CFG.AMP):
            pred = model(x3, li)
            if pred.shape[-1] != CFG.T_TRAIN:
                pred = F.interpolate(pred.unsqueeze(1), size=CFG.T_TRAIN, mode="linear", align_corners=False).squeeze(1)
        pred = torch.nan_to_num(pred, nan=0.0, posinf=0.0, neginf=0.0)
        y    = torch.nan_to_num(y,    nan=0.0, posinf=0.0, neginf=0.0)
        with torch.cuda.amp.autocast(enabled=False):
            loss = main_loss(pred.float(), y.float())
        losses.append(loss.item())
    return float(np.mean(losses))

best_val=1e9; best_path="ecgnet_v4m_plus_best.pt"
for ep in range(1, CFG.EPOCHS_MAIN+1):
    tr=train_one_epoch(); va=valid_epoch()
    print(f"[Main {ep}/{CFG.EPOCHS_MAIN}] train={tr:.5f}  valid={va:.5f}")
    if CFG.SAVE_BEST and va<best_val:
        best_val=va
        torch.save(model.state_dict(), best_path); print("  -> saved", best_path)
    torch.cuda.empty_cache(); gc.collect()

if os.path.exists(best_path):
    model.load_state_dict(torch.load(best_path, map_location=device)); print("Loaded best weights.")
torch.cuda.empty_cache(); gc.collect()

# ---------------------------
# Filters / helpers (fixed 15 Hz; stable)
# ---------------------------
def lowpass(sig, fs, order=2, cutoff_hz=15.0):
    fs = int(fs)
    sig = np.asarray(sig, dtype=np.float32)
    nyq = 0.5 * fs
    w = min(cutoff_hz / nyq, 0.999)
    b,a = scipy.signal.butter(order, w, btype='low')
    if len(sig) > 3 * max(len(a), len(b)):
        try:
            y = scipy.signal.filtfilt(b, a, sig).astype(np.float32)
            return np.nan_to_num(y, nan=0.0, posinf=0.0, neginf=0.0)
        except:
            pass
    return np.nan_to_num(sig, nan=0.0, posinf=0.0, neginf=0.0)

def scale_to_range(x, a=0.0, b=0.09):
    x = np.asarray(x, dtype=np.float32)
    xmin, xmax = float(np.min(x)), float(np.max(x))
    if not np.isfinite(xmin) or not np.isfinite(xmax):
        return np.full_like(x, (a+b)/2, dtype=np.float32)
    if xmax - xmin < 1e-8:
        return np.full_like(x, (a+b)/2, dtype=np.float32)
    y = (x - xmin) / (xmax - xmin)
    return (a + y * (b - a)).astype(np.float32)

# ---------------------------
# NCC + Safe dynamic blend (lag-align + sign-check)
# ---------------------------
def ncc(a, b):
    a = np.asarray(a, dtype=np.float32); b=np.asarray(b, dtype=np.float32)
    if len(a)!=len(b):
        b = resample_1d(b, len(a))
    ac = a - a.mean(); bc = b - b.mean()
    da = float(np.sqrt((ac*ac).sum()) + 1e-8)
    db = float(np.sqrt((bc*bc).sum()) + 1e-8)
    return float((ac*bc).sum() / (da*db))

def safe_blend(w_net, w_tpl, fs):
    # inputs same length & unscaled
    w_net = np.asarray(w_net, dtype=np.float32)
    w_tpl = np.asarray(w_tpl, dtype=np.float32)
    w_net = np.nan_to_num(w_net, nan=0.0, posinf=0.0, neginf=0.0)
    w_tpl = np.nan_to_num(w_tpl, nan=0.0, posinf=0.0, neginf=0.0)

    # Lag search within ±0.2s
    max_shift = int(fs * 0.2)
    a = (w_net - w_net.mean()) / (w_net.std() + 1e-8)
    b = (w_tpl - w_tpl.mean()) / (w_tpl.std() + 1e-8)
    corr = scipy.signal.correlate(a, b, mode='full')
    lags = scipy.signal.correlation_lags(len(a), len(b), mode='full')
    mask = (lags >= -max_shift) & (lags <= max_shift)
    lag = int(lags[mask][np.argmax(corr[mask])])

    if lag > 0:
        w_net_aligned = np.concatenate([np.full(lag, w_net[0], dtype=np.float32), w_net[:-lag]])
    elif lag < 0:
        lag2 = -lag
        w_net_aligned = np.concatenate([w_net[lag2:], np.full(lag2, w_net[-1], dtype=np.float32)])
    else:
        w_net_aligned = w_net

    # Sign check
    r = ncc(w_net_aligned, w_tpl)
    r_flip = ncc(-w_net_aligned, w_tpl)
    if r_flip > r:
        w_net_aligned = -w_net_aligned
        r = r_flip

    # Convert correlation to weight (template default)
    if r < 0.15:
        beta = 0.0
    else:
        beta = CFG.ENSEMBLE_BETA_GAIN * min(1.0, max(0.0, r))
        beta = float(np.clip(beta, CFG.ENSEMBLE_BETA_CLIP[0], CFG.ENSEMBLE_BETA_CLIP[1]))

    return beta, w_net_aligned

# ---------------------------
# Predict 12 leads for one image (ROI ensemble + tilt-aware TTA)
# ---------------------------
@torch.no_grad()
def predict_lead_waves_for_image(img01, roi_modes=("grid","unet"), roi_net_use=True):
    # detect tilt
    base_tilt = estimate_tilt_deg(img01)
    ttas = [t+base_tilt for t in CFG.TTA_BASE_DEGS]
    if CFG.TTA_EXTRA:
        ttas += [base_tilt-0.8, base_tilt+0.8]
    preds=[]
    for deg in ttas:
        im=rotate_image(img01,deg)
        for mode in roi_modes:
            use_net = (mode=="unet") and roi_net_use and (roi_net is not None)
            crops = get_crops(im, roi_mode=("unet" if use_net else mode), roi_net=(roi_net if use_net else None))
            xs=[stack_crop_to_x3(c) for c in crops]
            X=torch.tensor(np.stack(xs),dtype=torch.float32).to(device)
            li=torch.arange(12, device=device)
            if CFG.CHANNELS_LAST: X=X.to(memory_format=torch.channels_last)
            with torch.cuda.amp.autocast(enabled=CFG.AMP):
                out=model(X, li)  # (12,W)
            preds.append(out.detach().cpu().numpy())
    return np.mean(np.stack(preds,axis=0),axis=0).astype(np.float32)  # (12,W)

# ---------------------------
# Per-lead sign/gain calibration (small subset) — robust median
# ---------------------------
@torch.no_grad()
def compute_lead_calibration(sample_ids, max_images=32):
    gains = {lead: [] for lead in LEADS}
    for base_id in tqdm(list(sample_ids)[:max_images], desc="Calibrating (lead gains)"):
        p = pick_image_path(base_id, True)
        if p is None: 
            continue
        img01 = load_img_gray(p)
        waves = predict_lead_waves_for_image(img01, roi_modes=CFG.ROI_MODE_TESTS, roi_net_use=True)  # (12, Wpred)
        fs = int(train_csv.loc[train_csv.id==base_id, 'fs'].values[0])
        M  = load_wave_csv_matrix(f"{CFG.DATA_DIR}/train/{base_id}/{base_id}.csv")
        for lead in LEADS:
            li = LEAD_TO_IDX[lead]
            L  = int(fs*10) if lead=="II" else int(fs*2.5)
            gt = resample_1d(M[li], L).astype(np.float32)
            pr = resample_1d(waves[li], L).astype(np.float32)
            gtc = gt - gt.mean()
            prc = pr - pr.mean()
            denom = float((prc**2).sum()) + 1e-6
            a = float((prc*gtc).sum() / denom)
            gains[lead].append(a)
    lead_gain = {}
    for lead, arr in gains.items():
        if len(arr)==0:
            lead_gain[lead] = 1.0
        else:
            med = float(np.median(np.clip(arr, -3.0, 3.0)))
            lead_gain[lead] = float(np.clip(med, -2.5, 2.5))
    return lead_gain

calib_ids = list(train_ids)[:32]
LEAD_GAIN = compute_lead_calibration(calib_ids, max_images=32)
print("Per-lead gains:", {k: round(v,3) for k,v in LEAD_GAIN.items()})

# ---------------------------
# Templates
# ---------------------------
def build_lead_templates(template_len=500, max_samples=4000, cache_path=CFG.TEMPLATE_CACHE):
    if os.path.exists(cache_path):
        with open(cache_path, "rb") as f:
            obj = pickle.load(f)
        print("Loaded templates from cache.")
        return obj["templates"], obj["counts"]

    counts = {l:0 for l in LEADS}
    sums   = {l:np.zeros(template_len, dtype=np.float64) for l in LEADS}

    ids = train_csv["id"].values
    np.random.shuffle(ids)
    used = 0
    for base_id in tqdm(ids, total=min(len(ids), max_samples), desc="Building lead templates"):
        if used >= max_samples: break
        csv_path = f"{CFG.DATA_DIR}/train/{base_id}/{base_id}.csv"
        if not os.path.exists(csv_path): continue
        try:
            df = pd.read_csv(csv_path)
        except:
            continue
        used += 1
        for lead in LEADS:
            if lead not in df.columns: continue
            s = pd.to_numeric(df[lead], errors="coerce").dropna().values.astype(np.float32)
            if len(s) < 50: continue
            s = (s - s.mean()) / (s.std() + 1e-8)
            s_resamp = np.interp(np.linspace(0,1,template_len), np.linspace(0,1,len(s)), s)
            sums[lead] += s_resamp.astype(np.float64)
            counts[lead] += 1

    templates = {}
    for lead in LEADS:
        if counts[lead] > 0:
            templates[lead] = (sums[lead] / counts[lead]).astype(np.float32)
        else:
            t = np.linspace(0,1,template_len, dtype=np.float32)
            templates[lead] = np.sin(2*np.pi*t).astype(np.float32)

    with open(cache_path, "wb") as f:
        pickle.dump({"templates": templates, "counts": counts}, f)
    print("Saved templates to cache.")
    return templates, counts

LEAD_TEMPLATES, TEMPLATE_COUNTS = build_lead_templates(
    template_len=CFG.TEMPLATE_LEN,
    max_samples=CFG.TEMPLATE_MAX_SAMPLES,
    cache_path=CFG.TEMPLATE_CACHE
)
print("Template counts:", TEMPLATE_COUNTS)

# ---------------------------
# Local SNR sanity (Lead II)
# ---------------------------
if CFG.DO_LOCAL_SNR:
    try:
        model.eval()
        sample_ids = list(train_ids)[:4]
        rows_sol, rows_sub = [], []
        for base_id in tqdm(sample_ids, desc="Local SNR (Lead II only)"):
            p = pick_image_path(base_id, True)
            if p is None: continue
            img01 = load_img_gray(p)
            waves = predict_lead_waves_for_image(img01, roi_modes=CFG.ROI_MODE_TESTS, roi_net_use=True)  # (12, Wpred)

            fs = int(train_csv[train_csv.id == base_id]['fs'].values[0])
            M  = load_wave_csv_matrix(f"{CFG.DATA_DIR}/train/{base_id}/{base_id}.csv")

            lead = "II"; li = LEAD_TO_IDX[lead]; L = int(fs * 10)
            gt = resample_1d(M[li], L)
            pr = waves[li].astype(np.float32)
            pr = LEAD_GAIN.get(lead, 1.0) * pr
            pr = resample_1d(pr, L)

            # lowpass & scale for quick local check
            pr = lowpass(pr, fs, order=2, cutoff_hz=15.0)
            pr = scale_to_range(pr, CFG.OUT_MIN, CFG.OUT_MAX)

            for j in range(L):
                rid = f"{base_id}_{j}_{lead}"
                rows_sol.append((rid, fs, gt[j]))
                rows_sub.append((rid, pr[j]))
        sol = pd.DataFrame(rows_sol, columns=["id","fs","value"])
        sub = pd.DataFrame(rows_sub, columns=["id","value"])
        local_snr = competition_score(sol, sub, "id")
        print("Local SNR (Lead II only, dB):", round(local_snr, 4))
    except Exception as e:
        print("Local SNR eval skipped:", repr(e))

# ---------------------------
# Test Inference + Safe Template Blend + Submission
# ---------------------------
model.eval()
rows=[]
for base_id in tqdm(test_csv["id"].unique(), desc="Test inference"):
    p=f"{CFG.DATA_DIR}/test/{base_id}.png"
    if not os.path.exists(p): continue
    img01=load_img_gray(p)

    # network prediction (12 x Wpred)
    waves=predict_lead_waves_for_image(img01, roi_modes=CFG.ROI_MODE_TESTS, roi_net_use=True)

    subset=test_csv[test_csv["id"]==base_id]
    for _,r in subset.iterrows():
        lead=r["lead"]; fs=int(r["fs"]); nrows=int(r["number_of_rows"])
        li=LEAD_TO_IDX[lead]

        # --- network path (unscaled first) ---
        w = waves[li].astype(np.float32)
        w = LEAD_GAIN.get(lead, 1.0) * w
        w = np.nan_to_num(w, nan=0.0, posinf=0.0, neginf=0.0).astype(np.float32)
        w = resample_1d(w, nrows)  # resample to per-lead length

        # --- template path (unscaled) ---
        tpl = LEAD_TEMPLATES.get(lead, LEAD_TEMPLATES['II']).copy().astype(np.float32)
        tpl = np.interp(np.linspace(0,1,nrows), np.linspace(0,1,len(tpl)), tpl).astype(np.float32)

        # --- safe blend (align + sign safety; template default) ---
        beta, w_aligned = safe_blend(w, tpl, fs)

        # --- Low-pass & scale after alignment ---
        w_final   = scale_to_range(lowpass(w_aligned, fs, order=2, cutoff_hz=15.0), CFG.OUT_MIN, CFG.OUT_MAX)
        tpl_final = scale_to_range(lowpass(tpl,       fs, order=2, cutoff_hz=15.0), CFG.OUT_MIN, CFG.OUT_MAX)

        final = beta * w_final + (1.0 - beta) * tpl_final
        final = np.clip(final, CFG.OUT_MIN, CFG.OUT_MAX).astype(np.float32)

        for j in range(nrows):
            rows.append((f"{base_id}_{j}_{lead}", float(final[j])))

# --- Write submissions to Kaggle working output and verify ---
out_dir = "/kaggle/working"
os.makedirs(out_dir, exist_ok=True)

submission = pd.DataFrame(rows, columns=["id","value"]).astype({"id":"string","value":"float32"})
submission_parquet = os.path.join(out_dir, "submission.parquet")
submission_csv     = os.path.join(out_dir, "submission.csv")
submission.to_parquet(submission_parquet, index=False)
submission.to_csv(submission_csv, index=False)

print("✅ Saved:", submission_parquet, submission_csv)
print("CWD:", os.getcwd())
print("Working dir listing:", os.listdir(out_dir)[:20])
print(submission.head())
print("✅ Saved submission.parquet; rows:", len(submission))

# ===========================
# Submission validation + OFFLINE LB estimate (official metric)
# ===========================
print("\n=== Submission validation ===")
sub = submission.copy()
# Basic checks
n_ids_expected = len(test_csv)
n_rows_expected = int(test_csv['number_of_rows'].sum())
print("Unique test ECGs:", n_ids_expected)
print("Expected rows:", n_rows_expected, " | Submission rows:", len(sub))

assert len(sub) == n_rows_expected, "Row count mismatch."
assert sub['id'].is_unique, "IDs are not unique."
assert sub['value'].isna().sum() == 0, "NaNs in submission."
assert np.isfinite(sub['value']).all(), "Non-finite values in submission."
assert sub['value'].between(CFG.OUT_MIN - 1e-6, CFG.OUT_MAX + 1e-6).all(), "Values out of range."
print("✅ Submission looks valid.")

# ---------------------------
# OFFLINE LB estimate on held-out train (uses the official scoring functions)
# ---------------------------
from typing import Tuple
import scipy.optimize, scipy.signal

LEADS = ['I','II','III','aVR','aVL','aVF','V1','V2','V3','V4','V5','V6']
MAX_TIME_SHIFT = 0.2
PERFECT_SCORE = 384

class ParticipantVisibleError(Exception): pass

def compute_power(label: np.ndarray, prediction: np.ndarray) -> Tuple[float, float]:
    if label.ndim != 1 or prediction.ndim != 1:
        raise ParticipantVisibleError('Inputs must be 1-dimensional arrays.')
    finite_mask = np.isfinite(prediction)
    if not np.any(finite_mask):
        raise ParticipantVisibleError("The 'prediction' array contains no finite values (all NaN or inf).")
    prediction = prediction.copy()
    prediction[~np.isfinite(prediction)] = 0
    noise = label - prediction
    p_signal = np.sum(label**2)
    p_noise = np.sum(noise**2)
    return p_signal, p_noise

def compute_snr(signal: float, noise: float) -> float:
    if noise == 0:
        return PERFECT_SCORE
    if signal == 0:
        return 0
    return min((signal / noise), PERFECT_SCORE)

def align_signals(label: np.ndarray, pred: np.ndarray, max_shift: float = float('inf')) -> np.ndarray:
    if np.any(~np.isfinite(label)):
        raise ParticipantVisibleError('values in label should all be finite')
    if np.sum(np.isfinite(pred)) == 0:
        raise ParticipantVisibleError('prediction can not all be infinite')
    label_arr = np.asarray(label, dtype=np.float64)
    pred_arr = np.asarray(pred, dtype=np.float64)
    label_mean = np.mean(label_arr)
    pred_mean = np.mean(pred_arr)
    label_arr_centered = label_arr - label_mean
    pred_arr_centered = pred_arr - pred_mean
    correlation = scipy.signal.correlate(label_arr_centered, pred_arr_centered, mode='full')
    n_label = np.size(label_arr)
    n_pred = np.size(pred_arr)
    lags = scipy.signal.correlation_lags(n_label, n_pred, mode='full')
    valid_lags_mask = (lags >= -max_shift) & (lags <= max_shift)
    max_correlation = np.nanmax(correlation[valid_lags_mask])
    all_max_indices = np.flatnonzero(correlation == max_correlation)
    best_idx = min(all_max_indices, key=lambda i: abs(lags[i]))
    time_shift = lags[best_idx]
    start_padding_len = max(time_shift, 0)
    pred_slice_start = max(-time_shift, 0)
    pred_slice_end = min(n_label - time_shift, n_pred)
    end_padding_len = max(n_label - n_pred - time_shift, 0)
    aligned_pred = np.concatenate((np.full(start_padding_len, np.nan), pred_arr[pred_slice_start:pred_slice_end], np.full(end_padding_len, np.nan)))
    def objective_func(v_shift):
        return np.nansum((label_arr - (aligned_pred - v_shift)) ** 2)
    if np.any(np.isfinite(label_arr) & np.isfinite(aligned_pred)):
        results = scipy.optimize.minimize_scalar(objective_func, method='Brent')
        vertical_shift = results.x
        aligned_pred -= vertical_shift
    return aligned_pred

def _calculate_image_score(group: pd.DataFrame) -> float:
    unique_fs_values = group['fs'].unique()
    if len(unique_fs_values) != 1:
        raise ParticipantVisibleError('Sampling frequency should be consistent across each ecg')
    sampling_frequency = unique_fs_values[0]
    if sampling_frequency != int(len(group[group['lead']=='II']) / 10):
        raise ParticipantVisibleError('The sequence_length should be sampling frequency * 10s')
    sum_signal = 0.0
    sum_noise = 0.0
    for lead in LEADS:
        sub = group[group['lead'] == lead]
        label = sub['value_true'].values
        pred  = sub['value_pred'].values
        aligned_pred = align_signals(label, pred, int(sampling_frequency * MAX_TIME_SHIFT))
        p_signal, p_noise = compute_power(label, aligned_pred)
        sum_signal += p_signal
        sum_noise  += p_noise
    return compute_snr(sum_signal, sum_noise)

def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    for df in [solution, submission]:
        if row_id_column_name not in df.columns:
            raise ParticipantVisibleError(f"'{row_id_column_name}' column not found in DataFrame.")
        if df['value'].isna().any():
            raise ParticipantVisibleError('NaN exists in solution/submission')
        if not np.isfinite(df['value']).all():
            raise ParticipantVisibleError('Infinity exists in solution/submission')
    submission = submission[['id', 'value']]
    merged_df = pd.merge(solution, submission, on=row_id_column_name, suffixes=('_true', '_pred'))
    merged_df['image_id'] = merged_df[row_id_column_name].str.split('_').str[0]
    merged_df['row_id']   = merged_df[row_id_column_name].str.split('_').str[1].astype('int64')
    merged_df['lead']     = merged_df[row_id_column_name].str.split('_').str[2]
    merged_df.sort_values(by=['image_id','row_id','lead'], inplace=True)
    image_scores = merged_df.groupby('image_id').apply(_calculate_image_score, include_groups=False)
    return max(float(10 * np.log10(image_scores.mean())), -PERFECT_SCORE)

# ---- Build an offline validation set from held-out train (small sample) ----
np.random.seed(CFG.SEED)
val_eval_ids = list(val_ids)[:12]   # evaluate ~12 images for speed

rows_sol, rows_sub = [], []
for base_id in tqdm(val_eval_ids, desc="Offline LB (val subset)"):
    ip = pick_image_path(base_id, True)
    if ip is None: 
        continue
    img01 = load_img_gray(ip)
    waves = predict_lead_waves_for_image(img01, roi_modes=CFG.ROI_MODE_TESTS, roi_net_use=False)  # model inference only; blend below

    fs = int(train_csv.loc[train_csv.id==base_id, 'fs'].values[0])
    M  = load_wave_csv_matrix(f"{CFG.DATA_DIR}/train/{base_id}/{base_id}.csv")

    for lead in LEADS:
        li = LEAD_TO_IDX[lead]
        nrows = int(fs * (10 if lead == "II" else 2.5))

        # network (unscaled)
        w = waves[li].astype(np.float32)
        w = LEAD_GAIN.get(lead, 1.0) * w
        w = resample_1d(np.nan_to_num(w, nan=0.0, posinf=0.0, neginf=0.0), nrows)

        # template (unscaled)
        tpl = LEAD_TEMPLATES.get(lead, LEAD_TEMPLATES['II']).copy().astype(np.float32)
        tpl = np.interp(np.linspace(0,1,nrows), np.linspace(0,1,len(tpl)), tpl).astype(np.float32)

        # safe blend
        beta, w_aligned = safe_blend(w, tpl, fs)
        w_final   = scale_to_range(lowpass(w_aligned, fs, order=2, cutoff_hz=15.0), CFG.OUT_MIN, CFG.OUT_MAX)
        tpl_final = scale_to_range(lowpass(tpl,       fs, order=2, cutoff_hz=15.0), CFG.OUT_MIN, CFG.OUT_MAX)
        pred = np.clip(beta * w_final + (1.0 - beta) * tpl_final, CFG.OUT_MIN, CFG.OUT_MAX).astype(np.float32)

        # GT
        gt = resample_1d(M[li], nrows).astype(np.float32)

        for j in range(nrows):
            rows_sol.append((f"{base_id}_{j}_{lead}", fs, float(gt[j])))
            rows_sub.append((f"{base_id}_{j}_{lead}", float(pred[j])))

sol_df = pd.DataFrame(rows_sol, columns=["id","fs","value"])
sub_df = pd.DataFrame(rows_sub, columns=["id","value"])
try:
    offline_snr_db = score(sol_df, sub_df, "id")
    print(f"📈 Offline LB estimate (val subset, dB): {offline_snr_db:.4f}")
except Exception as e:
    print("Offline scoring skipped:", repr(e))